# 从一个典型的BP神经网络介绍TensorFlow中的各种操作

In [1]:
# 下面是读取数据和初始化的基本操作，不用介绍
from __future__ import print_function
import tensorflow as tf
import numpy as np

# 设置日志级别只是ERROR及以上，避免输出一大堆WARNING
tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# 学习参数和网络参数

learning_rate = 0.001
batch_size = 256
display_step = 100
num_steps = 3000

n_hidden_1 = 256
n_hidden_2 = 256
num_input = 28 *28
num_classes = 10

In [3]:
X = tf.placeholder(dtype=tf.float32, shape=[None, num_input])
Y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])

In [4]:
Weights = {
    'hidden_1':tf.Variable(tf.random_normal(shape=[num_input, n_hidden_1])),
    'hidden_2':tf.Variable(tf.random_normal(shape=[n_hidden_1, n_hidden_2])),
    'out':tf.Variable(tf.random_normal(shape=[n_hidden_2, num_classes]))
}

biases = {
    'hidden_1':tf.Variable(tf.random_normal(shape=[n_hidden_1])),
    'hidden_2':tf.Variable(tf.random_normal(shape=[n_hidden_2])),
    'out':tf.Variable(tf.random_normal(shape=[num_classes]))
}

In [5]:
def neural_net(x):
    layer1 = tf.add(tf.matmul(x, Weights['hidden_1']), biases['hidden_1'])
    layer2 = tf.add(tf.matmul(layer1, Weights['hidden_2']), biases['hidden_2'])
    out_layer = tf.add(tf.matmul(layer2, Weights['out']), biases['out'])
    return out_layer

In [6]:
# 定义训练
logits = neural_net(X)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate)

train_op = optimizer.minimize(loss_op)

# 定义测试准确率和损失
corrected_pred = tf.equal(tf.argmax(input=logits, axis=1), tf.argmax(input=Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(corrected_pred, tf.float32))

# 初始化全局变量
init_op = tf.global_variables_initializer()

In [7]:
# 开始训练
with tf.Session() as sess:
    sess.run(init_op)
    
    for step in range(1, num_steps + 1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x, Y:batch_y})
            
            print("Step {}, Minibatch loss is {:.4f}, Accuracy is {:.3f}".format(step, loss, acc))
            
    print("Optimize Finished!")
    
    acc = sess.run(accuracy, feed_dict={X:mnist.test.images[:1000], Y: mnist.test.labels[:1000]})
    print("Last accuracy is {:.4f}".format(acc))

Step 1, Minibatch loss is 3969.6597, Accuracy is 0.078
Step 100, Minibatch loss is 367.5314, Accuracy is 0.715
Step 200, Minibatch loss is 205.7923, Accuracy is 0.809
Step 300, Minibatch loss is 115.6662, Accuracy is 0.840
Step 400, Minibatch loss is 134.3223, Accuracy is 0.848
Step 500, Minibatch loss is 91.2530, Accuracy is 0.871
Step 600, Minibatch loss is 109.9074, Accuracy is 0.859
Step 700, Minibatch loss is 96.4995, Accuracy is 0.863
Step 800, Minibatch loss is 91.4111, Accuracy is 0.867
Step 900, Minibatch loss is 108.7307, Accuracy is 0.824
Step 1000, Minibatch loss is 62.4677, Accuracy is 0.887
Step 1100, Minibatch loss is 74.5396, Accuracy is 0.879
Step 1200, Minibatch loss is 62.9036, Accuracy is 0.891
Step 1300, Minibatch loss is 51.7317, Accuracy is 0.910
Step 1400, Minibatch loss is 71.3864, Accuracy is 0.898
Step 1500, Minibatch loss is 31.2562, Accuracy is 0.902
Step 1600, Minibatch loss is 43.5522, Accuracy is 0.867
Step 1700, Minibatch loss is 48.2615, Accuracy is 0.

这里好像除了优化的步骤外，其他的没有需要介绍。
1. 字典的定义完全是Python的数据类型
2. 其他的矩阵和向量的运算都是tf中的常规运算

tf.nn.softmax_cross_entropy_with_logits是个什么玩意儿，好像是用来计算交叉熵软回归API。关于它的介绍在[官网链接](https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/softmax_cross_entropy_with_logits)
关于Softmax Regression的介绍在[UFLDL](http://ufldl.stanford.edu/wiki/index.php/Softmax%E5%9B%9E%E5%BD%92)

tf.reduce_mean是降低维度，并求均值，没有指定维度信息，默认是求解所有元素的均值。

